# Practical Example: Document Scanning
- Goal: Extract text from a scanned document with uneven lighting.
- Steps:
    1. Convert to grayscale.
    2. Apply Gaussian blur to reduce noise.
    3. Use adaptive thresholding to create a binary image, separating text (black) from the background (white).
    4. Optionally, apply edge detection to refine text boundaries.

In [21]:
import cv2
import numpy as np

# Load image in grayscale
img_path = '../human_faces_and_object_dataset/Images/Objects/object_2002.png'
image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# Check if image loaded correctly
if image is None:
    print("Error: Could not load image. Check the file path.")
    exit()

# Step 1: Upscale image if text is too small (adjust scale_factor as needed)
scale_factor = 2
image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)

# Step 2: Enhance contrast using CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
contrast_enhanced = clahe.apply(image)

# Step 3: Apply light Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(contrast_enhanced, (3, 3), sigmaX=0.5)

# Step 4: Apply Otsu's thresholding
_, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Step 5: Morphological operations to clean up text
kernel = np.ones((1, 1), np.uint8)
binary_cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)  # Remove small noise
binary_cleaned = cv2.dilate(binary_cleaned, kernel, iterations=1)  # Slightly thicken text

# Step 6: Mild sharpening to enhance text edges
sharpen_kernel = np.array([[0, -1, 0], 
                          [-1, 5, -1], 
                          [0, -1, 0]])
sharpened = cv2.filter2D(binary_cleaned, -1, sharpen_kernel)

# Display results
cv2.imshow('Original', image)
cv2.imshow('Contrast Enhanced', contrast_enhanced)
cv2.imshow('Otsu Thresholded and Sharpened', sharpened)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save result
# cv2.imwrite('sharpened_text_otsu.jpg', sharpened)